In [5]:
import pandas as pd
from pathlib import Path
import sqlite3
import glob
import json
import datetime
import os
from tqdm import tqdm

from ollama import Client

In [6]:
def test_analysis_market():
    # SYSTEM TEMPLATE
    system_templates = [
        # Corto
        (
            "Eres un asistente de investigación del sector energético colombiano. "
            "Trabaja solo con fuentes públicas (CREG, ANLA, Superintendencia, datos.gov.co, cámaras de comercio, prensa especializada, LinkedIn). "
            "Extrae datos clave (finanzas, licencias, noticias, proyectos, tecnología, sostenibilidad) y cita fuente y fecha."
        ),

        # Medio
        (
            "Eres un asistente de investigación especializado en el sector energético colombiano. "
            "Usa exclusivamente fuentes públicas y accesibles en Internet, como portales regulatorios (CREG, ANLA, Superintendencia), cámaras de comercio, medios especializados y LinkedIn. "
            "Debes recopilar y estructurar, indicando fuente y fecha, la siguiente información:\n"
            " 1. Estados financieros (ingresos, utilidad neta)\n"
            " 2. Concesiones y licencias vigentes\n"
            " 3. Noticias relevantes de los últimos 12 meses\n"
            " 4. Portafolio de proyectos\n"
            " 5. Tecnologías de datos y plataformas usadas\n"
            " 6. Indicadores de sostenibilidad\n"
            "Entrega un informe organizado por secciones."
        ),

        # Largo
        (
            "Eres un asistente de investigación para el sector energético colombiano, con enfoque en información 100 % pública y verificable. "
            "Tu labor es buscar en portales oficiales (CREG, ANLA, Superintendencia de Servicios Públicos, Ministerio de Minas y Energía), "
            "en el Registro Único de Proponentes y cámaras de comercio, en medios especializados (Energía Estratégica, Portafolio, La República), "
            "en plataformas de datos abiertos (datos.gov.co) y en redes profesionales (LinkedIn).\n\n"
            "Para cada empresa, extrae y estructura los siguientes elementos, siempre indicando la fuente y la fecha de publicación:\n"
            " 1. Estados financieros públicos (ingresos anuales, utilidad neta)\n"
            " 2. Concesiones, licencias o permisos vigentes (fechas de otorgamiento y expiración)\n"
            " 3. Principales noticias de los últimos 12 meses (titulares, fechas, temas)\n"
            " 4. Portafolio de proyectos en operación o asignados\n"
            " 5. Alianzas y joint ventures anunciadas\n"
            " 6. Tecnologías, plataformas y herramientas de análisis de datos mencionadas\n"
            " 7. Indicadores de sostenibilidad y RSE\n"
            " 8. APIs públicas o iniciativas de datos abiertos\n\n"
            "Devuélvelo en un informe claro, dividido por secciones, con enlaces activos y fechas precisas."
        )
    ]


    # SYSTEM TEMPLATE
    prompt_templates = [
        # 1. Estados financieros públicos
        "Busca el último informe financiero o balance público de '{company}' y extrae sus ingresos anuales y utilidad neta. "
        "Indica el año del informe y la fuente (portal de la Superintendencia o web corporativa).",

        # 2. Registro y concesiones
        "Consulta el Registro Único de Proponentes, la ANLA o la CREG para identificar las concesiones, licencias o permisos vigentes de '{company}'. "
        "Enumera las fechas de otorgamiento y expiración de cada permiso.",

        # 3. Noticias y prensa del mercado
        "Revisa medios especializados (e.g. Energía Estratégica, Portafolio) y prensa local para listar las principales noticias en los últimos 12 meses sobre '{company}'. "
        "Resume titulares, fechas y temas (expansión, sanciones, proyectos nuevos).",

        # 4. Perfil corporativo y propiedad
        "Extrae del sitio web de la Cámara de Comercio y la Superintendencia de Servicios Públicos la estructura societaria de '{company}': "
        "¿Pública, privada o mixta? Indica porcentaje de participación pública según datos disponibles.",

        # 5. Portafolio de proyectos
        "En la web de '{company}' o en comunicados de prensa, identifica los proyectos energéticos (plantas, líneas de transmisión, parques renovables) asignados o en operación. "
        "Anota nombres, ubicaciones y capacidad instalada.",

        # 6. Alianzas y joint ventures
        "Busca comunicados oficiales o notas de prensa donde '{company}' anuncie alianzas, joint ventures o asociaciones estratégicas. "
        "Enumera socios, fecha de anuncio y objetivo de la alianza.",

        # 7. Tecnología y digitalización (fuentes públicas)
        "Revisa el sitio web, blog o LinkedIn de '{company}' para listar las tecnologías, plataformas o herramientas de análisis de datos que mencionen como parte de su oferta o procesos internos.",

        # 8. Presencia en redes profesionales
        "En LinkedIn, busca vacantes o perfiles asociados a '{company}' con roles en datos, BI o TI. "
        "Cuenta cuántas posiciones activas hay y qué competencias piden, para inferir su madurez digital.",

        # 9. Indicadores de sostenibilidad y responsabilidad social
        "Localiza el informe de sostenibilidad o RSE (si existe) de '{company}' en su página web o en Global Reporting Initiative. "
        "Extrae los principales indicadores (emisiones, inversión social) y fecha de publicación.",

        # 10. Políticas de datos abiertos o API públicas
        "Consulta si '{company}' participa en iniciativas de datos abiertos (datos.gov.co) o si publica APIs/portales de datos. "
        "Proporciona enlaces y describe brevemente qué datasets o servicios ofrecen."
    ]

    companies = [
        'COMPAÑIA DE ELECTRICIDAD DE TULUA S.A. E.S.P.',
        'A.S.C. INGENIERIA S.A. E.S.P.',
        'EMPRESA DE ENERGIA DEL PUTUMAYO S.A. E.S.P.',
        'FOTOVOLTAICO ARRAYANES S.A.S. E.S.P.',
        'ENERTOTAL S.A. E.S.P.'
    ]

    test_dir = Path('test')
    test_dir.mkdir(exist_ok=True)

    for i, company in tqdm(enumerate(companies), desc="Procesando empresas", total=len(companies)):
        for j, prompt_template in tqdm(enumerate(prompt_templates), desc=f"Procesando prompts para {company}", total=len(prompt_templates)):
            client = Client(host="http://localhost:11434", timeout=None)
            messages = [
                {"role": "system", "content": system_template},
                {"role": "user", "content": prompt_template.format(company=company)}
            ]
            response = client.chat(model="gemma3:latest", messages=messages)
            # Guardar la respuesta en un archivo JSON
            response_data = {
                "company": company,
                "system_template": system_template,
                "prompt": prompt_template.format(company=company),
                "response": response.message.content,
                "date": datetime.datetime.now().isoformat()
            }
            with open(test_dir / f"{i:03}_{j:03}.json", "w") as f:
                json.dump(response_data, f, indent=4)


In [9]:
pd.concat([pd.read_json(file, lines=True) for file in glob.glob('test/*.json')])

ValueError: Expected object or value